# Decision Tree Algorithm
Decision trees can be applied to both regression and classification problems.

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

%matplotlib inline

In [2]:
dummy_train_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]
headers = ['color','diameter','fruit']

## CART(Classification and Regression Trees)

## Question(Parition concept of the dataset)

took from google example

In [3]:
class Question:
    
    def __init__(self,column_index,value,header):
        self.column_index = column_index
        self.value = value 
        self.header = header
        
    def match(self, example):
        """
        Notes:
            1. here example can be a numpy array of a list of size (size,)
                because we are dealing with single row iteration for tree
        
        """
        
        if isinstance(example,list):
            example = np.array(example,dtype="O")
        val = example[self.column_index]
        
        if isinstance(val,(int,float,np.int64,np.float64)): # adding numpy int and float data types as well
            return float(val) >= float(self.value) # a condition for question to return True or False for numeric value
        else:
            return str(val) == str(self.value) # categorical data comparison
        
    def __repr__(self):
        
        condition = "=="
        if isinstance(self.value,(int,float,np.int64,np.float64)):
            condition = ">="
            
        return f"Is {self.header} {condition} {self.value} ?"       
        

In [4]:
q1 = Question(column_index=0,value='apple',header='fruit')
q1

Is fruit == apple ?

In [5]:
q1.match(['apple',10])

True

In [6]:
q2 = Question(column_index=1,value=30,header='weight')
q2

Is weight >= 30 ?

In [7]:
q2.match(['apple',10])

False

## Partition rows

In [8]:
def partition(rows,question):
    
    true_side, false_side = [],[]
    for row in rows:
        if question.match(row):
            true_side.append(row)
        else:
            false_side.append(row)
    return true_side, false_side

In [9]:
partition(dummy_train_data,Question(column_index=0,value='Red',header=headers[0]))

([['Red', 1, 'Grape'], ['Red', 1, 'Grape']],
 [['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']])

## Gini Impurity

Used by the CART (classification and regression tree) algorithm for classification trees, Gini impurity is a measure of how often a randomly chosen element from the set would be incorrectly labeled if it was randomly labeled according to the distribution of labels in the subset. The Gini impurity can be computed by summing the probability $p_{i}$ of an item with label i being chosen times the probability 

$\sum_{k \neq i}{p_k} = 1 - {p_i}$

of a mistake in categorizing that item. It reaches its minimum (zero) when all cases in the node fall into a single target category.

$$
{I_G(p)} = {\sum_{i=1}^{J}{(p_i\sum_{k \neq i}p_k)}} = {\sum_{i=1}^{J}p_i(1 - p_i)} = {\sum_{i=1}^{J}(p_i - p_i^2)} = {\sum_{i=1}^{J}p_i - \sum_{i=1}^{J}p_i^2} = { 1 - \sum_{i=1}^{J}p_i^2} 
$$



In [10]:
def gini_impurity(y):
    """
    References:
        https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    if isinstance(y,(list,tuple)):
        y = np.array(y,dtype='O')
        
    if len(y.shape) == 1:
        y = y.copy().reshape(-1,1)
    
    sum_val = 0
    classes = np.unique(y[:,-1])
    for cl in classes:
        p = len(np.where(y == cl)[0])/ y.shape[0]
        sum_val += p**2    
    gini_score = 1 - sum_val
    return gini_score

In [11]:
print(gini_impurity(np.array([0,0,0,0,0,0,0,1,1,1,1,1,1])))

0.4970414201183432


In [12]:
print(gini_impurity(np.array(['apple','apple','apple','apple'])))

0.0


In [13]:
print(gini_impurity(np.array(['apple','apple','apple','orange'])))

0.375


In [14]:
print(gini_impurity(np.array(['apple','apple','apple','orange','orange','orange'])))

0.5


In [15]:
print(gini_impurity(['apple','apple','apple','orange','orange','orange','lemon']))

0.6122448979591837


## Entropy

In [16]:
def entropy(y):
    if isinstance(y,(list,tuple)):
        y = np.array(y,dtype='O')
        
    if len(y.shape) == 1:
        y = y.copy().reshape(-1,1)
    
    entropy_score = 0
    classes = np.unique(y)
    for cl in classes:
        p = len(np.where(y == cl)[0])/ y.shape[0]
        entropy = -p * np.log2(p)
        entropy_score += entropy
        
    return entropy_score

In [17]:
print(entropy(np.array([0,0,0,0,0,0,0,1,1,1,1,1,1])))

0.9957274520849255


In [18]:
print(entropy(np.array(['apple','apple','apple','apple'])))

0.0


In [19]:
print(entropy(np.array(['apple','apple','apple','orange'])))

0.8112781244591328


In [20]:
print(entropy(np.array(['apple','apple','apple','orange','orange','orange'])))

1.0


In [21]:
print(entropy(['apple','apple','apple','orange','orange','orange','lemon','lemon','apple']))

1.5304930567574826


## Information Gain

In [22]:
def info_gain(left_side, right_side, current_uncertainity):
    
    if isinstance(left_side,(list,tuple)):
        left_side = np.array(left_side,dtype='O')
    
    if isinstance(right_side,(list,tuple)):
        right_side = np.array(right_side,dtype='O')
    
    pr = left_side.shape[0] / (left_side.shape[0] + right_side.shape[0])
    
    info_gain_value = current_uncertainity - pr * gini_impurity(left_side) - ( 1 - pr ) * gini_impurity(right_side)
    
    return info_gain_value

In [23]:
headers,dummy_train_data

(['color', 'diameter', 'fruit'],
 [['Green', 3, 'Apple'],
  ['Yellow', 3, 'Apple'],
  ['Red', 1, 'Grape'],
  ['Red', 1, 'Grape'],
  ['Yellow', 3, 'Lemon']])

In [24]:
curr_uncert = gini_impurity(dummy_train_data)
curr_uncert

0.6399999999999999

In [25]:
ts , fs = partition(dummy_train_data,Question(0,'Green',headers[0]))

In [26]:
ts

[['Green', 3, 'Apple']]

In [27]:
fs

[['Yellow', 3, 'Apple'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape'],
 ['Yellow', 3, 'Lemon']]

In [28]:
info_gain(ts,fs,curr_uncert)

0.1399999999999999

In [29]:
ts,fs = partition(dummy_train_data,Question(0,'Red',headers[0]))

In [30]:
ts

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

In [31]:
fs 

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

In [32]:
info_gain(ts,fs,curr_uncert)

0.37333333333333324

## Find best split in all options

In [33]:
def find_best_split(rows,headers):
    
    if not isinstance(rows,np.ndarray):
        rows = np.array(rows,dtype='O')
    
    max_gain = 0
    best_split_question = None
    
    current_inconsistency = gini_impurity(rows)
    n = rows.shape[1] - 1 
    
    for col_index in range(n):
        
        unique_values = np.unique(rows[:,col_index])
        for val in unique_values:
            
            ques = Question(column_index=col_index,value=val,header=headers[col_index])
            
            true_side,false_side = partition(rows,ques)
            
            if len(true_side) == 0 or len(false_side) == 0:
                continue
                
            gain = info_gain(true_side, false_side, current_inconsistency)
            
            if gain >= max_gain:
                max_gain,best_split_question = gain ,ques
    return max_gain, best_split_question            

In [34]:
find_best_split(dummy_train_data,headers)

(0.37333333333333324, Is diameter >= 3 ?)

## Leaf node

In [66]:
class LeafNode:
    def __init__(self, rows):
        self.predictions = dict(zip(*np.unique(np.array(rows,dtype='O')[:,-1],return_counts=True)))


## Decision Node

In [36]:
class DecisionNode:
    
    def __init__(self,question,true_branch,false_branch):
        
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

## Build Tree

In [37]:
def build_tree(rows,headers):
    
    if not isinstance(rows,np.ndarray):
        rows = np.array(rows,dtype='O')
    
    gain, ques = find_best_split(rows,headers)
    
    if gain == 0:
        return LeafNode(rows)

    true_rows,false_rows = partition(rows,ques)
    
    true_branch = build_tree(true_rows,headers)
    false_branch = build_tree(false_rows,headers)
    
    return DecisionNode(ques,true_branch,false_branch)

## Print tree

In [38]:
def print_tree(node, spacing=""):
    
    if isinstance(node,LeafNode):
        print(spacing," Predict", node.predictions)
        return
    
    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")


In [39]:
my_tree = build_tree(dummy_train_data,headers)

[['Yellow' 3 'Apple']
 ['Yellow' 3 'Lemon']]
{'Apple': 1, 'Lemon': 1}
[['Green' 3 'Apple']]
{'Apple': 1}
[['Red' 1 'Grape']
 ['Red' 1 'Grape']]
{'Grape': 2}


In [40]:
print_tree(my_tree)

Is diameter >= 3 ?
--> True:
  Is color == Yellow ?
  --> True:
      Predict {'Apple': 1, 'Lemon': 1}
  --> False:
      Predict {'Apple': 1}
--> False:
    Predict {'Grape': 2}


## Classification

In [41]:
def classification(row,node):
    
    if isinstance(node,LeafNode):
        return node.predictions
    
    if node.question.match(row):
        return classification(row,node.true_branch)
    else:
        return classification(row,node.false_branch)

In [42]:
dummy_train_data

[['Green', 3, 'Apple'],
 ['Yellow', 3, 'Apple'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape'],
 ['Yellow', 3, 'Lemon']]

In [43]:
classification(dummy_train_data[0],my_tree)

{'Apple': 1}

In [44]:
classification(dummy_train_data[1],my_tree)

{'Apple': 1, 'Lemon': 1}

## Print leaf node

In [45]:
def print_leaf(results):
    total = sum(results.values()) 
    
    probs = {}
    for key in results:
        probs[key] = (results[key] / total) * 100
    return probs

In [46]:
print_leaf(classification(dummy_train_data[0],my_tree))

{'Apple': 100.0}

In [47]:
print_leaf(classification(dummy_train_data[1],my_tree))

{'Apple': 50.0, 'Lemon': 50.0}

In [67]:
from sklearn.datasets import load_iris

dataset = load_iris()

rows = np.hstack((dataset.data,dataset.target.reshape(-1,1)))
headers = dataset.feature_names + ['target']

my_tree = build_tree(rows,headers)

In [69]:
print_tree(my_tree)

Is petal width (cm) >= 1.0 ?
--> True:
  Is petal width (cm) >= 1.8 ?
  --> True:
    Is sepal length (cm) >= 5.8 ?
    --> True:
      Is sepal length (cm) >= 7.9 ?
      --> True:
          Predict {2.0: 1}
      --> False:
        Is petal length (cm) >= 4.9 ?
        --> True:
          Is petal width (cm) >= 2.1 ?
          --> True:
              Predict {2.0: 23}
          --> False:
            Is petal width (cm) >= 2.0 ?
            --> True:
                Predict {2.0: 3}
            --> False:
                Predict {2.0: 14}
        --> False:
          Is sepal width (cm) >= 3.2 ?
          --> True:
              Predict {1.0: 1}
          --> False:
              Predict {2.0: 2}
    --> False:
        Predict {2.0: 2}
  --> False:
    Is petal width (cm) >= 1.1 ?
    --> True:
      Is petal length (cm) >= 5.0 ?
      --> True:
        Is petal width (cm) >= 1.6 ?
        --> True:
          Is petal length (cm) >= 5.8 ?
          --> True:
              Predict {2.

In [88]:
print_leaf(classification(rows,my_tree))

{0.0: 100.0}